In [1]:
args = dict(
	model='FasterDecoding/medusa-1.0-vicuna-7b-v1.5',
	temperature=0.7,
	max_steps=512,
	style='simple',
	multiline=False,
	mouse=False,
	debug=False,
)


In [2]:
import torch

from medusa.model.medusa_model import MedusaModel


model = MedusaModel.from_pretrained(
    args['model'],
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto",
)
model

/home/camus/work/Medusa/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/camus/work/Medusa/env/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/camus/work/Medusa/env/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embe

MedusaModelLlama(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [3]:
tokenizer = model.get_tokenizer()
tokenizer

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


LlamaTokenizerFast(name_or_path='FasterDecoding/medusa-1.0-vicuna-7b-v1.5', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>', 'additional_special_tokens': ['<unk>', '<s>', '</s>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [4]:
from fastchat.model.model_adapter import get_conversation_template


conv = get_conversation_template(args['model'])
conv

Conversation(name='vicuna_v1.1', system_template='{system_message}', system_message="A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.", roles=('USER', 'ASSISTANT'), messages=[], offset=0, sep_style=<SeparatorStyle.ADD_COLON_TWO: 2>, sep=' ', sep2='</s>', stop_str=None, stop_token_ids=None)

In [7]:
conv.roles

('USER', 'ASSISTANT')

In [6]:
from fastchat.serve.cli import SimpleChatIO


chatio = SimpleChatIO(args['multiline'])

inp = chatio.prompt_for_input(conv.roles[0])
inp

'hello'

In [8]:
conv.append_message(conv.roles[0], inp)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()

prompt

"A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: hello ASSISTANT:"

In [15]:
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.base_model.device)
input_ids.shape, input_ids

(torch.Size([1, 40]),
 tensor([[    1,   319, 13563,  1546,   263, 12758,  1404,   322,   385, 23116,
          21082, 20255, 29889,   450, 20255,  4076,  8444, 29892, 13173, 29892,
            322,  1248,   568,  6089,   304,   278,  1404, 29915, 29879,  5155,
          29889,  3148,  1001, 29901, 22172,   319,  1799,  9047, 13566, 29901]],
        device='cuda:0'))

In [16]:
gen = model.medusa_generate(
    input_ids,
    temperature=args['temperature'],
    max_steps=args['max_steps'],
)

it = iter(gen)
next(it)

{'text': 'Hello! How can I'}

In [17]:
next(it)

{'text': 'Hello! How can I help you today?'}